# Session 2: Track Alignment and Data Loading

Welcome to the second part of the hands-on tutorial! In this notebook, we:

- Introduce additional building blocks from the Gosling visualization grammer (track composition primitives) 

- Showcase the transparent data-utilties from **`gos`** to help users visualize their own local (and in-memory) genomics datasets

To get started, make sure you have `gosling` installed. 

In [ ]:
!pip install gosling[all]==0.0.9
import gosling as gos


> **Note**: be sure to include the `[all]` for this notebook when installing `gos` from PyPI via `pip`. The special data utilites in `gos` are opt-in by default, and running the install command as shown above ensures that you will have the correct dependencies. 

# Track Composition

A `gos.Track` is a unit building block which can be represented as a bar chart, a line chart, ideogram, or other chart types as we've seen in the first session. For the concurrent analysis of multiple datasets, multiple _Tracks_ can be grouped into one or more _Views_ and navigated synchronously. Each `gos.View` defines the genomic location for every `gos.Track` it contains, and each `gos.Track` binds and maps data to be visualized.

Changes following View-level properties modifies how _Tracks_ and _Views_ compose together:

- `layout` - Whether to view genomic positions in Cartesian (`linear`) or in polar (`circular`) coordinate systems.

- `alignment` - Whether multiple tracks should `overlay` or `stack` within a _single_ View.

- `arrangement` - How to juxtapose _multiple_ Views together (`parallel`, `serial`, `vertical`, `horizontal`).

This section will walk through each of these properties in more detail with examples.

## `layout` - Linear and Circular

The _View_ `layout` property specifies whether genomic coordinates are represented in either a `circular` or `linear` layout.

The following figure displays the upper _Track_ with a `linear` layout and the bottom with a `circular` layout.

<p align="center">
  <img width="600" src="https://github.com/gosling-lang/gosling-website/raw/main/static/img/doc_images/linear_circular.png" />
</p>

Here we create a bar chart visualization displaying a "pseudobulk" excitatory neuron scATAC-seq track from [Corces et. al (_Nature Genetics, 2020_)](https://www.nature.com/articles/s41588-020-00721-x), and apply a `linear` and `circular` layout.

In [ ]:
data = gos.bigwig(
    url="https://s3.amazonaws.com/gosling-lang.org/data/ExcitatoryNeurons-insertions_bin100_RIPnorm.bw",
    column="position", 
    value="peak",
)

track = gos.Track(data, height=50).mark_bar().encode(
    x=gos.X("position:G"),
    y=gos.Y("peak:Q", axis="right"),
)

track.view()

### Linear view

In [ ]:
track.view(layout="linear")

### Circular view

In [ ]:
track.properties(width=200).view(layout="circular")

## `alignment` -  Multiple Tracks in One View​

The _View_ `alignment` property allows users to either `"overlay"` or `"stack"` several tracks.

<p align="center">
  <img width="600" src="https://github.com/gosling-lang/gosling-website/raw/main/static/img/doc_images/alignment.png" />
</p>


When setting `alignment` as `"overlay"`, multiple tracks are layered on top of others. When setting `alignment as "stack", multiple tracks are vertically concantenated. The default value of alignment is "stack".

In [ ]:
line = track.mark_line().encode(color=gos.value("#f3b285"))

bar = track.encode(color=gos.value("#4472c4"))

print(line, bar)

### Stacked tracks in single view

In [ ]:
gos.stack(bar, line, line).properties(spacing=1)

In [ ]:
gos.stack(
    bar.properties(width=400),
    line.properties(width=400),
).properties(spacing=1, layout="circular")

### Overlay tracks in single view

In [ ]:
gos.stack(bar, line)

In [ ]:
gos.overlay(
    bar.properties(width=400),
    line.properties(width=400),
).properties(spacing=1, layout="circular")

### Example: Gene Annotation

The `overlay` alignment is an essential feature in Gosling and allows much more complex visual encodings. We leverage this building-block in the code example below to build a gene annotation track, combining several primitive tracks.

> **Note** its not essential that you understand every detail of the code below. It is provided to demonstrate more complex usage of the `gos` API and can be used for reference later.

In [ ]:
import gosling as gos

# HiGlass-based annotation dataset
# http://gosling-lang.org/docs/data#beddb-require-higlass-server
genes = gos.beddb(
    url="https://server.gosling-lang.org/api/v1/tileset_info/?d=gene-annotation",
    genomicFields=[
      {"index": 1, "name": "start"},
      {"index": 2, "name": "end"}
    ],
    valueFields=[
      {"index": 5, "name": "strand", "type": "nominal"},
      {"index": 3, "name": "name", "type": "nominal"}
    ],
    exonIntervalFields=[
      {"index": 12, "name": "start"},
      {"index": 13, "name": "end"}
    ]
)

# Primitive Tracks

base = gos.Track(genes).encode(
    row=gos.Row("strand:N", domain=["+", "-"]),
    color=gos.Color("strand:N", domain=["+", "-"], range=["#7585FF", "#FF8A85"]),
).properties(
    height=100,
    title="Genes | hg38",
)

plusGeneHead = base.mark_triangleRight(align="left").encode(
    x=gos.X("end:G", axis="top"),
    size=gos.value(15)
).transform_filter(
    field="type", oneOf=["gene"]
).transform_filter(
    field="strand", oneOf=["+"]
)

minusGeneHead = base.mark_triangleLeft(align="right").encode(
    x=gos.X("start:G", axis="top"),
    size=gos.value(15)
).transform_filter(
    field="type", oneOf=["gene"]
).transform_filter(
    field="strand", oneOf=["-"]
)

geneLabel = base.mark_text(dy=15).encode(
    x=gos.X("start:G", axis="top"),
    xe="end:G",
    text="name:N",
    size=gos.value(15)
).transform_filter(
    field="type", oneOf=["gene"]
).visibility_lt(
    measure="width",
    threshold="|xe-x|",
    transitionPadding=10,
    target="mark",
)

exon = base.mark_rect().encode(
    x=gos.X("start:G", axis="top"),
    xe="end:G",
    size=gos.value(15)
).transform_filter(
    field="type", oneOf=["exon"]
)

plusGeneRange = base.mark_rule(linePattern={"type": "triangleRight", "size": 5}).encode(
    x=gos.X("start:G", axis="top"),
    xe="end:G",
    strokeWidth=gos.value(3),
).transform_filter(
    field="type", oneOf=["gene"]
).transform_filter(
    field="strand", oneOf=["+"]
)

minusGeneRange = base.mark_rule(linePattern={"type": "triangleLeft", "size": 5}).encode(
    x=gos.X("start:G", axis="top"),
    xe="end:G",
    strokeWidth=gos.value(3),
).transform_filter(
    field="type", oneOf=["gene"]
).transform_filter(
    field="strand", oneOf=["-"]
)

# Combine tracks with overlay alignment

gene_annotation = gos.overlay(
    plusGeneRange, minusGeneRange, exon, plusGeneHead, minusGeneHead, geneLabel
)

gene_annotation.properties(
    xDomain=gos.GenomicDomain(chromosome="1", interval=[103400000, 103700000]),
)

We can then "stack" our custom `gene_annotation` view with the scATAC-seq track from earlier to provide context.

In [ ]:
gos.stack(
    gene_annotation,
    bar.properties(title="Excitatory Neuron ATAC-seq"),
).properties(
    xDomain=gos.GenomicDomain(chromosome="1", interval=[103400000, 103700000]),
)

## `arrangement` - Arrange Multiple Views​

Gosling supports multi-view visualizations. Users specify the visualizations under the `views` property and modify the arragement of these fields through the `arrangement` property. 

`gos` provides several top-level utilities to conveniently group multiple views together in each of the arrangement options:

- `gos.horizontal`
- `gos.vertical`
- `gos.serial`
- `gos.parallel`

<p align="center">
 <img width="600" src="https://github.com/gosling-lang/gosling-website/raw/main/static/img/doc_images/multi_views.png" />
</p>

> **Note** the arrangement pairs (`gos.serial`, `gos.horizontal`) and (`gos.parallel`, `gos.vertical`) are **equivalent** when combining _Views_ with `linear` layouts. Behavior _only_ differs when arranging `circular` layout _Views_ (left-most column).

### `gos.horizontal` & `gos.vertical`

We can compose `gos.horizontal` and `gos.vertical` to arrange multiple tracks as into separate views. Notice how interactions are no longer synchronized since each _Track_ is within a unique _View_.

In [ ]:
gos.horizontal(
    # left
    gos.stack(
        track.properties(width=300, height=100),
        track.properties(width=300, height=100),
    ),
    # right
    track.encode(color=gos.value("#4472c4")).properties(width=600, height=240)
)

We can selectively "group" tracks within the **same** view using the `alignment` features in the previous section. We replace `gos.vertical` with `gos.stack`, which combines the tracks on the left into the same _View_."

In [ ]:
gos.horizontal(
    # left
    gos.stack(
        gene_annotation.properties(width=300, height=100),
        track.properties(width=300, height=100)
    ),
    # right
    track.encode(color=gos.value("#4472c4")).properties(width=600, height=240)
)

### `gos.serial` & `gos.parallel`

The `serial` and `parallel` alignments differ in behavior from `horizontal` and `veritcal` when combining _Views_ with `circular` alignments.

In [ ]:
gos.serial(
    track.properties(width=300).view(layout="circular"), 
    gene_annotation.properties(width=300, layout="circular"),
)

# Loading data

This section illustrates a key (optional) feature of `gos` which makes hosting data for your Gosling visualizations a breeze. 

Normally a Gosling visualization requires the administration of a web-server to host both the client and genomics data sets for the visualization. In `gos`, we provide further integration with Python to hide this complexity and allow remote, local, and in-memory data to be visualized seamlessly through an idential API.

In this notebook, we will visualize the same [BED file](https://samtools.github.io/hts-specs/BEDv1.pdf) containing h38 cytoband information as a: 

- remote dataset (via URL) 
- local dataset (via local path)
- in memory (from a `pd.DataFrame`)

## The visualization

The `ideogram` function generates an ideogram visualization for a given Gosling data source. It is not important that you understand the details of this block to follow along in this notebook. Moreover, the important bit is to understand that `ideogram` takes `data` as input and returns a Gosling visualization created with the `gos` API.

We will show how this function can be _reused_ for various `data` defintions (genomic data sources).

In [ ]:
def ideogram(data) -> gos.View:
    track = gos.Track(data) # bind data to track
    
    arms = track.mark_rect().encode(
        color=gos.Color("stain:N",
            domain=["gneg", "gpos25", "gpos50", "gpos75", "gpos100", "gvar"],
            range=["white", "#D9D9D9", "#979797", "#636363", "black", "#A0A0F2"],
        ),
        x=gos.X("chromStart:G", axis="none"),
        xe="chromEnd:G",
        stroke=gos.value("black"),
        strokeWidth=gos.value(0.5),
    ).transform_filter_not(
        field="stain",
        oneOf=["acen"],
    )

    labels = track.mark_text().encode(
        text="name:N",
        color=gos.Color("stain:N",
            domain=["gneg", "gpos25", "gpos50", "gpos75", "gpos100", "gvar"],
            range=["black", "#636363", "black", "#D9D9D9", "white", "black"],
        ),
        strokeWidth=gos.value(0)
    ).visibility_lt(
        target='mark',
        measure='width',
        threshold='|xe-x|',
        transitionPadding=10
    )

    centromere = track.encode(
        x=gos.X("chromStart:G"),
        xe="chromEnd:G",
        color=gos.value('red'),
    ).transform_filter(
        "stain", oneOf=["acen"]
    )

    centromere_left = centromere.mark_triangleLeft().transform_filter(
        "name", include="p"
    )

    centromere_right = centromere.mark_triangleRight().transform_filter(
        "name", include="q"
    )

    return gos.overlay(arms, labels, centromere_left, centromere_right).properties(height=20)


## The dataset

The `url` below links to a [BED4+1](https://samtools.github.io/hts-specs/BEDv1.pdf) file containing UCSC hg38 cytoband information. This dataset is hosted on GitHub and is avaiable via URL. 

In [ ]:
url = "https://raw.githubusercontent.com/sehilyi/gemini-datasets/master/data/UCSC.HG38.Human.CytoBandIdeogram.bed"

In [ ]:
!curl -s {url}  | head | column -t
# chrom  chromStart  chromEnd  name  stain

## Remote dataset (via URL)

We can reference this URL directly in Gos by creating a CSV data source via `gos.csv(...)`. This function returns a Python dictionary that describes our dataset to Gosling. We use the `gos.csv` utility since the resource is a columnar text dataset.

In [ ]:
# specify BED4+1 format
data = gos.csv(
    url=url,
    headerNames=['chrom', 'chromStart', 'chromEnd', 'name', 'stain'], # the +1 field is stain
    chromosomeField="chrom", # the column containing chrom names
    genomicFields=["chromStart", "chromEnd"], # fields with genomic coordinates
    separator="\t",
)

data

We can now pass this dataset directly to the `ideogram` function which binds `data` to `gos.Track` and creates our custom visualization.

In [ ]:
ideogram(data)

This visualization is a bit crowded since we are viewing the data genome-wide. We can set the initial genomic domain for the visualization to Chromosome 2 by specifying `xDomain` as a property.

In [ ]:
ideogram(data).properties(xDomain=gos.GenomicDomain(chromosome="chr2"))

## Local Dataset (via local filepath)

Data are not always publically available via URL like above, and often we'd like to visualize local data files. To visualize local data, **simply change the URL to a local file path**.

```diff
data = gos.csv(
-  url=url,
+  url="./UCSC.HG38.Human.CytoBandIdeogram.bed",
   ... 
)
```

Below we download the file from GitHub and load the visualization from our local filesytem.

In [ ]:
!wget {url} # download file

In [ ]:
!cat UCSC.HG38.Human.CytoBandIdeogram.bed | head | column -t # print local file contents

In [ ]:
!ls

In [ ]:
data = gos.csv(
    url="./UCSC.HG38.Human.CytoBandIdeogram.bed",
    # url=url
    headerNames=['chrom', 'chromStart', 'chromEnd', 'name', 'stain'],
    chromosomeField="chrom",
    genomicFields=["chromStart", "chromEnd"],
    separator="\t",
)

# reuse the same visualization
ideogram(data).properties(xDomain=gos.GenomicDomain(chromosome="chr2"))

## In memory (via `pd.DataFrame`)

While loading remote and local genomics data files is useful, often we want to visualize intermediate or derived information during analysis. Rather than writing these results to disk, `gos` supports visualizing in-memory data directly from Pandas dataframes `pd.DataFrame`.

In order to use this feature, we first load our dataset as a `pd.DataFrame`.

In [ ]:
import pandas as pd

df = pd.read_csv(
    './UCSC.HG38.Human.CytoBandIdeogram.bed', 
    names=['chrom', 'chromStart', 'chromEnd', 'name', 'stain'],
    sep="\t"
)

df.head()

Lets filter `df` in Python for our dataset only contains entries for Chromosome 2.

In [ ]:
df = df[df.chrom == "chr2"]
df.head()

We can now create a `data` source for our visualization using the `df.gos.csv(...)` method, and visualize directly! Note how the resulting visualization only renders for chromosome 2.

In [ ]:
df.gos.csv(    # we only need to specify these fields since the rest are inferred from dataframe
    chromosomeField="chrom",
    genomicFields=["chromStart", "chromEnd"], 
)

In [ ]:
data = df.gos.csv(
    # we only need to specify these fields since the rest are inferred from dataframe
    chromosomeField="chrom",
    genomicFields=["chromStart", "chromEnd"], 
)

ideogram(data) # view in context of full assembly

In [ ]:
ideogram(data).properties(xDomain=gos.GenomicDomain(chromosome="chr2")) # view just chrom 2

# (BONUS) Exercise: local bigwig track

In the previous notebook we visualized several scATAC-seq "pseudobulk" tracks from human brain cells using a simple barplot.

The `bigwig_url` below points to a [BigWig file (161Mb)](https://s3.amazonaws.com/gosling-lang.org/data/ExcitatoryNeurons-insertions_bin100_RIPnorm.bw) on a **remote** server containing the normliazed aggregate signal from excitatory neurons cells in the human brain.

In [ ]:
bigwig_url = "https://s3.amazonaws.com/gosling-lang.org/data/ExcitatoryNeurons-insertions_bin100_RIPnorm.bw"

We can combine the `gene_annotation` _View_ from earlier with a track for our scATAC-seq dataset to provide context when visualizing the peaks.

In [ ]:
data_bw = gos.bigwig(url=bigwig_url, column="position", value="peak")

excitatory_neurons = gos.Track(data_bw).mark_bar().encode(
    x="position:G",
    y="peak:Q",
).properties(height=50)

gos.stack(gene_annotation, excitatory_neurons).properties(
    xDomain=gos.GenomicDomain(chromosome="1", interval=[103400000, 103700000]),
)

Data is not always available via public URL, and often will be stored locally during an analysis. 

**Task**: 

1. *download* the scATAC-seq dataset locally (**hint**: use `!wget` in the cell below)
1. *change* the code snippet above to load the _local_ `ExcitatoryNeurons-insertions_bin100_RIPnorm.bw`